In [1]:
from external_lib.gnss_derived import *
import pandas as pd
from tqdm.notebook import tqdm
import os
import numpy as np
import multiprocessing as mp
import scipy.optimize as opt
from lib.noglobal import noglobal

from lib.io import load_pickle_data
from scipy import optimize

### util function

In [2]:
%%time

@noglobal()
def ecef2lla(x, y, z):
    # x, y and z are scalars or vectors in meters
    x = np.array([x]).reshape(np.array([x]).shape[-1], 1)
    y = np.array([y]).reshape(np.array([y]).shape[-1], 1)
    z = np.array([z]).reshape(np.array([z]).shape[-1], 1)

    a=6378137
    a_sq=a**2
    e = 8.181919084261345e-2
    e_sq = 6.69437999014e-3

    f = 1/298.257223563
    b = a*(1-f)

    # calculations:
    r = np.sqrt(x**2 + y**2)
    ep_sq  = (a**2-b**2)/b**2
    ee = (a**2-b**2)
    f = (54*b**2)*(z**2)
    g = r**2 + (1 - e_sq)*(z**2) - e_sq*ee*2
    c = (e_sq**2)*f*r**2/(g**3)
    s = (1 + c + np.sqrt(c**2 + 2*c))**(1/3.)
    p = f/(3.*(g**2)*(s + (1./s) + 1)**2)
    q = np.sqrt(1 + 2*p*e_sq**2)
    r_0 = -(p*e_sq*r)/(1+q) + np.sqrt(0.5*(a**2)*(1+(1./q)) - p*(z**2)*(1-e_sq)/(q*(1+q)) - 0.5*p*(r**2))
    u = np.sqrt((r - e_sq*r_0)**2 + z**2)
    v = np.sqrt((r - e_sq*r_0)**2 + (1 - e_sq)*z**2)
    z_0 = (b**2)*z/(a*v)
    h = u*(1 - b**2/(a*v))
    phi = np.arctan((z + ep_sq*z_0)/r)
    lambd = np.arctan2(y, x)

    return phi*180/np.pi, lambd*180/np.pi, h


@noglobal()
def distance(sat_pos, x):
    sat_pos_diff = sat_pos.copy(deep=True)
                
    sat_pos_diff['xSatPosMRotated'] = sat_pos_diff['xSatPosMRotated'] - x[0]
    sat_pos_diff['ySatPosMRotated'] = sat_pos_diff['ySatPosMRotated'] - x[1]
    sat_pos_diff['zSatPosMRotated'] = sat_pos_diff['zSatPosMRotated'] - x[2]
    
    return sat_pos_diff["uncertaintyWeight"]*(np.sqrt((sat_pos_diff["xSatPosMRotated"]**2 + sat_pos_diff["ySatPosMRotated"]**2 + sat_pos_diff["zSatPosMRotated"]**2))+ x[3] - sat_pos_diff["correctedPrM"])

@noglobal()
def distance_fixed_satpos(x,train_df):    
    return distance(train_df[['xSatPosMRotated', 'ySatPosMRotated', 'zSatPosMRotated', 'correctedPrM', 'uncertaintyWeight']], x)

@noglobal()
def predict_by_satelite_per_time(list_):
    key = list_[0]
    each_df = list_[1]
    
    x0= [0,0,0,0]        
    opt_res = opt.least_squares(lambda x: distance_fixed_satpos(x,each_df), x0)
    
    opt_res_pos = opt_res["x"]
    wls_estimated_pos = ecef2lla(*opt_res_pos[:3])
    wls_estimated_pos = np.squeeze(wls_estimated_pos)
    return [key, wls_estimated_pos[0],wls_estimated_pos[1],wls_estimated_pos[2]]
    
@noglobal()
def multi(function_name,exe_list):
    p = mp.Pool(mp.cpu_count()-1);
    retult = p.map(function_name,exe_list);
    p.close();
    return retult    

@noglobal()
def predict_by_gnss_data(df_train):

            
    df_train['correctedPrM'] = df_train["rawPrM"] +  df_train["satClkBiasM"]  - df_train["isrbM"] - df_train["ionoDelayM"] - df_train["tropoDelayM"]
    
    ### Rotating the Satellite Reference Frame
    light_speed = 299_792_458  
    df_train['transmissionTimeSeconds'] = df_train['correctedPrM'] / light_speed    
    omega_e = 7.2921151467e-5
    df_train["xSatPosMRotated"] = np.cos(omega_e * df_train['transmissionTimeSeconds']) * df_train['xSatPosM'] + np.sin(omega_e * df_train['transmissionTimeSeconds']) * df_train['ySatPosM']
    df_train["ySatPosMRotated"] = - np.sin(omega_e * df_train['transmissionTimeSeconds']) * df_train['xSatPosM'] + np.cos(omega_e * df_train['transmissionTimeSeconds']) * df_train['ySatPosM']
    df_train['zSatPosMRotated'] = df_train['zSatPosM']
    
    #display(df_train)
    df_train['uncertaintyWeight'] = 1 / df_train['rawPrUncM']
    
    for key,each_df in df_train.groupby("millisSinceGpsEpoch"):        
        
        print(each_df.shape);
        
                
    test_list = [];        
    exe_list = [[key,each_df]  for key,each_df in df_train.groupby("millisSinceGpsEpoch")];        
    #num = multi(predict_by_satelite_per_time,exe_list)
    
                
    return pd.DataFrame(np.array(num),columns=["millisSinceGpsEpoch","x_pred","y_pred","z_pred"]);

    

#df_baseline = pd.read_csv("/work/data/input/google-smartphone-decimeter-challenge/baseline_locations_train.csv");    
#extracted_df = df_baseline[df_baseline["phone"] == collection+"_"+phone];

root = "/work/data/input/google-smartphone-decimeter-challenge/"
collection = "2020-05-29-US-MTV-1"
phone = "Pixel4"

file_path = f"{root}/train/{collection}/{phone}/{phone}_derived.csv"
derived_df = pd.read_csv(file_path)
display(derived_df.head(3))
gnss = predict_by_gnss_data(derived_df);

,collectionName,phoneName,millisSinceGpsEpoch,constellationType,svid,signalType,receivedSvTimeInGpsNanos,xSatPosM,ySatPosM,zSatPosM,xSatVelMps,ySatVelMps,zSatVelMps,satClkBiasM,satClkDriftMps,rawPrM,rawPrUncM,isrbM,ionoDelayM,tropoDelayM
0,2020-05-29-US-MTV-1,Pixel4,1274827487438,1,5,GPS_L1,1274827486368290474,768322.056,-2.264197e+07,1.356280e+07,929.619,1510.509,2463.883,-4080.658,-0.001,2.104596e+07,4.197,0.000,4.162,3.131
1,2020-05-29-US-MTV-1,Pixel4,1274827487438,1,2,GPS_L1,1274827486361182225,8482050.448,-1.431711e+07,2.133148e+07,2550.228,275.714,-860.292,-139060.060,0.000,2.317696e+07,8.094,0.000,5.406,4.542
2,2020-05-29-US-MTV-1,Pixel4,1274827487438,6,11,GAL_E1,1274827486357190690,8224045.037,-1.644270e+07,2.319549e+07,2305.626,-342.014,-1057.763,939972.639,0.070,2.437360e+07,8.094,-223.007,5.071,4.122


(21, 26)
(15, 26)
(16, 26)
(18, 26)
(20, 26)
(19, 26)
(16, 26)
(19, 26)
(19, 26)
(20, 26)
(15, 26)
(18, 26)
(13, 26)
(16, 26)
(19, 26)
(21, 26)
(21, 26)
(20, 26)
(21, 26)
(20, 26)
(20, 26)
(20, 26)
(19, 26)
(19, 26)
(20, 26)
(18, 26)
(17, 26)
(19, 26)
(20, 26)
(18, 26)
(21, 26)
(18, 26)
(18, 26)
(16, 26)
(15, 26)
(19, 26)
(19, 26)
(19, 26)
(23, 26)
(21, 26)
(21, 26)
(23, 26)
(18, 26)
(20, 26)
(18, 26)
(23, 26)
(27, 26)
(22, 26)
(24, 26)
(26, 26)
(24, 26)
(22, 26)
(20, 26)
(23, 26)
(22, 26)
(29, 26)
(26, 26)
(23, 26)
(27, 26)
(29, 26)
(28, 26)
(31, 26)
(31, 26)
(32, 26)
(32, 26)
(30, 26)
(32, 26)
(32, 26)
(31, 26)
(30, 26)
(31, 26)
(29, 26)
(30, 26)
(29, 26)
(29, 26)
(29, 26)
(29, 26)
(30, 26)
(31, 26)
(30, 26)
(26, 26)
(27, 26)
(29, 26)
(31, 26)
(30, 26)
(27, 26)
(28, 26)
(25, 26)
(24, 26)
(26, 26)
(25, 26)
(26, 26)
(28, 26)
(28, 26)
(28, 26)
(27, 26)
(25, 26)
(25, 26)
(27, 26)
(27, 26)
(27, 26)
(25, 26)
(25, 26)
(28, 26)
(29, 26)
(30, 26)
(27, 26)
(27, 26)
(28, 26)
(28, 26)
(31, 26)
(

NameError: name 'num' is not defined

In [3]:
from external_lib.gnss_transformer import gnss_log_to_dataframes

root = "/work/data/input/google-smartphone-decimeter-challenge/"
file_path = f"{root}/train/{collection}/{phone}/{phone}_derived.csv"
    
derived_df = pd.read_csv(file_path)

root = "/work/data/input/google-smartphone-decimeter-challenge/"
gnss_df = gnss_log_to_dataframes(f"{root}/train/{collection}/{phone}/{phone}_GnssLog.txt")
derived_df = pd.read_csv(f"{root}/train/{collection}/{phone}/{phone}_derived.csv")

In [4]:
from tqdm.notebook import tqdm




@noglobal()
def add_prev_info(derived_df):    
    s = derived_df.copy()
    time_stamp_prev_hashmap = {};
    millis_list = s["millisSinceGpsEpoch"].unique().tolist();
    for idx in range(len(millis_list)):    
        time_stamp_prev_hashmap[millis_list[idx]] = millis_list[max(idx-1,0)]
        
    s["prev_millisSinceGpsEpoch"] = s["millisSinceGpsEpoch"].apply(lambda x: time_stamp_prev_hashmap[x])
    s["time_delta_sec"] = (s["millisSinceGpsEpoch"] - s["prev_millisSinceGpsEpoch"])/1000
    s[['prev_ionoDelayM','prev_tropoDelayM',"prev_satClkBiasM"]] = s.shift(1)[['ionoDelayM','tropoDelayM',"satClkBiasM"]]
    return s;

@noglobal()
def add_prev_info_for_gnss_raw(gnss_df):
    s = gnss_df.copy();
    
    s["prev_"]
    
    

@noglobal()
def extract_between_time(df,prev_gpstime,current_gpstime):
    return df[(df["millisSinceGpsEpoch"]  >= prev_gpstime) & (df["millisSinceGpsEpoch"]  <current_gpstime)]

@noglobal()
def extract_equal_time(df,current_gpstime):
    return extract_between_time(df,current_gpstime,current_gpstime)

@noglobal()
def corrected_delta_adr_mps_add(derived_df,gnss_raw_df):
                    
    counter = 0;
    
    gps_time_list = derived_df["millisSinceGpsEpoch"].unique()
    
    #for idx in range(1,len(gps_time_list)):
        
    
    
    for key, each_df in tqdm(derived_df.groupby("millisSinceGpsEpoch")):
        if (not counter == 0):
            if (not len(each_df["time_delta_sec"].unique()) == 1):
                raise Exception("asdf");
            
            timedelta = each_df["time_delta_sec"].unique()[0];
                        
            prev,now = each_df.reset_index().loc[0,["prev_millisSinceGpsEpoch","millisSinceGpsEpoch"]]
            
            each_gnss_df = extract_between_time(gnss_raw_df,prev,now);
            
            eq_each_gnss_df = extract_equal_time(gnss_raw_df,prev)
            
            #eq_each_gnss_df
                        
            corrected_delta_adr_mps = each_gnss_df["AccumulatedDeltaRangeMeters"]
            
            corrected_delta_adr_mps[rows_with_adr_support] = (each_gnss_df["AccumulatedDeltaRangeMeters"].to_numpy() 
                                                              - each_gnss_df["prev_AccumulatedDeltaRangeMeters"].to_numpy())/delta_t_sec
            corrected_delta_adr_unc_mps[rows_with_adr_support] = df_cur_epoch.iloc[
                rows_with_adr_support, adrm_unc_col_idx] / delta_t_sec            
            
                        
            delta_atmos_delay_mps = (
                (each_df['ionoDelayM'] + each_df['tropoDelayM']).to_numpy() -
                (each_df['ionoDelayM'] +  each_df['tropoDelayM']).to_numpy()
            ) / timedelta
            
            clock_drift_mps = (each_df['satClkBiasM'].to_numpy() -
                    each_df['prev_satClkBiasM'].to_numpy()) / timedelta
            
            corrected_delta_adr_mps = corrected_delta_adr_mps + clock_drift_mps - delta_atmos_delay_mps
            
            
            
        counter = counter +1;
            
        
    
    


derived_df_with_prev = add_prev_info(derived_df.copy());    
gnss_df_with_date = UTC2GpsEpoch(gnss_df["Raw"].copy())
#derived_df_with_prev = corrected_delta_adr_mps_add(derived_df_with_prev,gnss_df_with_date)  
#display(derived_df)    

In [5]:
@noglobal()
def correct_millisSinceGpsEpoch(df_derived,df_raw_train):
    
    df_raw_train = UTC2GpsEpoch(df_raw_train)
    #df_raw_train['MillisSinceGpsEpoch'] = np.floor( (df_raw_train['TimeNanos'] - df_raw_train['FullBiasNanos']) / 1000000.0).astype(int)            
    
    
    df_derived_corrected = pd.DataFrame() 
    for indexes, subdf in df_derived.groupby(['collectionName', 'phoneName']):
        df_raw_sub = df_raw_train[(df_raw_train['collectionName']==indexes[0])&(df_raw_train['phoneName']==indexes[1])]
    
        # Change each value in df_derived['MillisSinceGpsEpoch'] to be the prior epoch.
        raw_timestamps = df_raw_sub['millisSinceGpsEpoch'].unique()
        derived_timestamps = subdf['millisSinceGpsEpoch'].unique()

        # The timestamps in derived are one epoch ahead. We need to map each epoch
        # in derived to the prior one (in Raw).
        indexes = np.searchsorted(raw_timestamps, derived_timestamps)
        from_t_to_fix_derived = dict(zip(derived_timestamps, raw_timestamps[indexes-1]))
        subdf['millisSinceGpsEpoch'] = np.array(list(map(lambda v: from_t_to_fix_derived[v], subdf['millisSinceGpsEpoch'])))

    df_derived_corrected = pd.concat([df_derived_corrected, subdf])
    return df_derived_corrected

In [6]:
mode = "train"
derived_df = load_derived_file(collection,phone,mode)
df_raw_train = load_gnss_raw_file(collection,phone,mode)
derived_df = correct_millisSinceGpsEpoch(derived_df,df_raw_train)

#display(df_raw_train)

In [7]:
copied_derived_df = derived_df.copy()

time_stamp_prev_hashmap = {};
millis_list = copied_derived_df["millisSinceGpsEpoch"].unique().tolist();
for idx in range(len(millis_list)):    
    time_stamp_prev_hashmap[millis_list[idx]] = millis_list[max(idx-1,0)]

copied_derived_df["prev_millisSinceGpsEpoch"] = copied_derived_df["millisSinceGpsEpoch"].apply(lambda x: time_stamp_prev_hashmap[x]);
copied_derived_df["time_delta_sec"] = (copied_derived_df["millisSinceGpsEpoch"] - copied_derived_df["prev_millisSinceGpsEpoch"])/1000
copied_derived_df[['prev_ionoDelayM','prev_tropoDelayM']] = copied_derived_df.shift(1)[['ionoDelayM','tropoDelayM']]

prev,now = copied_derived_df.reset_index().loc[700,["prev_millisSinceGpsEpoch","millisSinceGpsEpoch"]]

gnss_extracted_df = extract_between_time(gnss_row,prev,now);
gnss_extracted_df = gnss_extracted_df.sort_values("Svid")

derived_extracted_df = copied_derived_df[copied_derived_df["millisSinceGpsEpoch"] == prev]
derived_extracted_df = derived_extracted_df.sort_values("svid")

print(gnss_extracted_df.shape,derived_extracted_df.shape)

### Train

In [8]:
mode = "train"
total_derived_df = load_derived_df_all(mode)
total_gnss_df = load_gnss_df_all(mode)

100%|██████████| 73/73 [00:28<00:00,  2.59it/s]


In [9]:
bd_list = [];
for key,each_gnss_df in total_gnss_df.groupby("constellationName"):
        
    gnss_freq_list = sorted(each_gnss_df["carrierFrequencyHz"].astype(int).unique())
    #derived_svid_list = sorted(each_derived_df["CarrierFrequencyHz"].astype(int).unique())            
    for freq in gnss_freq_list:
        bd_list.append([key,freq,freq_and_consname_to_satelliteType(key,freq)]);

In [10]:
total_gnss_df.columns

Index(['utcTimeMillis', 'TimeNanos', 'LeapSecond', 'TimeUncertaintyNanos',
       'FullBiasNanos', 'BiasNanos', 'BiasUncertaintyNanos',
       'DriftNanosPerSecond', 'DriftUncertaintyNanosPerSecond',
       'HardwareClockDiscontinuityCount', 'svid', 'TimeOffsetNanos', 'State',
       'ReceivedSvTimeNanos', 'ReceivedSvTimeUncertaintyNanos', 'Cn0DbHz',
       'PseudorangeRateMetersPerSecond',
       'PseudorangeRateUncertaintyMetersPerSecond',
       'AccumulatedDeltaRangeState', 'AccumulatedDeltaRangeMeters',
       'AccumulatedDeltaRangeUncertaintyMeters', 'carrierFrequencyHz',
       'CarrierCycles', 'CarrierPhase', 'CarrierPhaseUncertainty',
       'MultipathIndicator', 'SnrInDb', 'constellationType', 'AgcDb',
       'collectionName', 'phoneName', 'phone', 'millisSinceGpsEpoch',
       'constellationName', 'signalType', 'BasebandCn0DbHz',
       'FullInterSignalBiasNanos', 'FullInterSignalBiasUncertaintyNanos',
       'SatelliteInterSignalBiasNanos',
       'SatelliteInterSignalBiasU

In [11]:
cons_name_freq_list = [ i+"_"+str(j) for i,j,k in bd_list];
satelite_type_list = [ k for i,j,k in bd_list];

dict_ = {};
for cons,sate_type in zip(cons_name_freq_list,satelite_type_list):
    dict_[cons] = sate_type
    dict_[cons+".0"] = sate_type

In [12]:
from lib.io import save_as_pickle
save_as_pickle(dict_,"/work/data/input/cons_name_freq_to_satelite_type_dict/dictionary.pkl")

Exception: The file already exists
/work/data/input/cons_name_freq_to_satelite_type_dict/dictionary.pkl

## test 

In [ ]:
mode = "test"
total_gnss_df = load_gnss_df(mode)
total_derived_df = load_derived_df(mode)
mapping_dict = load_constellation_mapping_file_as_dict()
total_gnss_df["ConstellationName"] = total_gnss_df["ConstellationType"].map(mapping_dict)
total_derived_df["ConstellationName"] = total_derived_df["constellationType"].map(mapping_dict)

In [ ]:
%%time
## QZZZ
new_to_old = {1:(183, 193), 2:(184, 194, 196), 3:(187, 189, 197, 199), 4:(185, 195, 200)}
old_to_new={}
for new_svid, old_svids in new_to_old.items():
    for s in old_svids:
        old_to_new[s] = new_svid  

total_gnss_df.loc[total_gnss_df["ConstellationType"] == 4,"Svid"] = total_gnss_df.loc[total_gnss_df["ConstellationType"] == 4,"Svid"].replace(old_to_new)

In [ ]:
for key,each_gnss_df in total_gnss_df.groupby("ConstellationName"):
    each_derived_df = total_derived_df[total_derived_df["ConstellationName"] == key];
    
    gnss_svid_list = sorted(each_gnss_df["Svid"].astype(int).unique())
    derived_svid_list = sorted(each_derived_df["svid"].astype(int).unique())        
    
    
    print("----------------------"+key+"----------------------")
    
    print(derived_svid_list)
    print(gnss_svid_list)
    for dev_svid in derived_svid_list:
        if (not dev_svid in gnss_svid_list):
            print(key,dev_svid)

## test 

In [ ]:
import glob 
from external_lib.gnss_transformer import gnss_log_to_dataframes,load_gnss_data_from_file
from tqdm.notebook import tqdm


FILES = glob.glob("/work/data/input/google-smartphone-decimeter-challenge/test/*/*/*_GnssLog.txt");
s = [];
for i in tqdm(FILES):    
    try:        
        each_df = gnss_log_to_dataframes(i)["Raw"]
        each_df["collectionName"] = i.split("/")[-3]
        each_df["phoneName"] = i.split("/")[-2]
        each_df["phone"] = each_df["collectionName"] + "_" + each_df["phoneName"]
        s.append(each_df)
    except Exception as e:
        print(i);

In [ ]:
df = load_constellation_mapping_file_as_dict()

total_gnss_df = pd.concat(s)
total_gnss_df["ConstellationName"] = total_gnss_df["ConstellationType"].map(df)

for key,each_df in total_gnss_df.groupby("ConstellationName"):
    print("----------",key,"----------")    
    print(sorted(each_df["Svid"].unique()))
    #print(each_df["Svid"].value_counts())

In [ ]:
FILES = glob.glob("/work/data/input/google-smartphone-decimeter-challenge/test/*/*/*_derived.csv");

pd_list = []
for f in FILES:
    derived_df = pd.read_csv(f)
    pd_list.append(derived_df)    
derived_df_list = pd.concat(pd_list);

In [ ]:
derived_df_list["constellationName"] = derived_df_list["constellationType"].map(df)

for key,each_df in derived_df_list.groupby("constellationName"):
    print("----------",key,"----------")    
    print(sorted(each_df["svid"].unique()))